Q)
1. 대지면적 0이면 Null처리해줘야 하는 것 아닌가?

In [12]:
import pandas as pd
import numpy as np

In [16]:
df = pd.read_csv('dataset.csv', engine='python')

In [18]:
df.head()

,시군구,유형,지번,도로명,용도지역,건축물주용도,도로조건,전용/연면적(㎡),대지면적(㎡),거래금액(만원),층,계약년월,계약일,건축년도
0,서울특별시강남구개포동,일반,1***,논현로,준주거,제2종근린생활,25m이상,381.42,367.3,"550,000",NaN,201708,23,1993.0
1,서울특별시강남구개포동,일반,1**,선릉로10길,준주거,제2종근린생활,8m미만,535.20,207.8,"390,000",NaN,201708,29,1987.0
2,서울특별시강남구개포동,집합,1*,개포로,일반상업,판매,25m이상,17.64,NaN,"39,500",1.0,201709,4,1997.0
3,서울특별시강남구개포동,집합,1**,개포로,일반상업,제2종근린생활,25m미만,82.70,NaN,"60,000",NaN,201709,14,1984.0
4,서울특별시강남구개포동,집합,1**,개포로,제3종일반주거,제1종근린생활,25m이상,9.00,NaN,"15,000",1.0,201709,27,1983.0


In [19]:
df.dtypes

시군구           object
유형            object
지번            object
도로명           object
용도지역          object
건축물주용도        object
도로조건          object
전용/연면적(㎡)    float64
대지면적(㎡)      float64
거래금액(만원)      object
층            float64
계약년월           int64
계약일            int64
건축년도         float64
dtype: object

In [26]:
df = df.replace(0,np.nan) # 0은 이상값이라 판단하여 제거함
df = df.replace('-',np.nan)#도로조건은 -가 null 값
df.isnull().sum()

시군구              0
유형               0
지번               0
도로명           1517
용도지역             0
건축물주용도           0
도로조건          1288
전용/연면적(㎡)        0
대지면적(㎡)      37910
거래금액(만원)         0
층            27463
계약년월             0
계약일              0
건축년도           842
dtype: int64

In [27]:
for i in df.columns :
    X_i = df.loc[:,i] 
    if (X_i.dropna().shape[0]/df.shape[0])!=1 :
        print("column {} : {} not null ratio".format(i,X_i.dropna().shape[0]/df.shape[0]))
        print(df.loc[:,i].unique())
        print("")
        print()

column 도로명 : 0.9676366429150488 not null ratio
['논현로' '선릉로10길' '개포로' ... '용마산로125길' '봉화산로15길' '중랑천로20길']


column 도로조건 : 0.97252208047105 not null ratio
['25m이상' '8m미만' '25m미만' '12m미만' nan]


column 대지면적(㎡) : 0.19123607970303366 not null ratio
[ 367.3  207.8    nan ...  521.  3974.6  811.2]


column 층 : 0.4141101676835772 not null ratio
[nan  1. 11. 13.  5.  4. 30.  2.  8.  9. 10.  7. 12.  3. 15. 25. 24. 32.
 18.  6. 14. 19. 16. 17. 20. 21. 22. 23. 26. 27. 28. 29. 34. 33. 35. 36.
 31. 60. 50. 61. 70. 80. 51. 40. 41. 47. 42. 44. 43. 45. 90. 71. 81. 82.
 62. 52. 72. 54. 46. 37. 91. 53. 63. 64.]


column 건축년도 : 0.9820369501216026 not null ratio
[1993. 1987. 1997. 1984. 1983. 1990. 2004. 1991. 1982. 1995. 1998. 2002.
 2017. 2003. 2006. 1986. 1980. 1978. 2009. 2001. 2010. 1989. 1994. 1985.
 1992. 2005. 1988. 2016. 1996. 1981. 2000. 1975. 2007. 2011. 1974. 1976.
 2014. 2018. 2008. 2015. 1979. 2012. 1999. 1977. 1968. 2013. 1972. 1970.
 1969. 1961. 1971. 1973. 1967. 1965. 1960. 1962.   nan 194

In [7]:
##형변환
#계약년월 to datetime64
import datetime
import math
contract_yr = []
for i, j in np.array(df[['계약년월', '계약일']]):
    contract_yr.append(datetime.datetime(int(str(int(i))[:4]), int(str(int(i))[4:6]), j))
df['계약년월_adj'] = contract_yr


#건축년도 to datetime64
df.fillna({'건축년도':0}, inplace=True) ##null값 처리 ,datetime엔 not null 값이 들어가야함

construct_yr =[]
for i in np.array(df['건축년도']):
    if(i==0):
        construct_yr.append(datetime.datetime(1900,1,1)) # null 값 1900년 1월 1일로 설정
    else :
        construct_yr.append(datetime.datetime(int(str(int(i))[:4]),1,1))                      
df['건축년도_adj']=construct_yr
df.head()

#거래금액 to float64
df['거래금액(만원)_adj']=df['거래금액(만원)'].str.replace(',', '').astype('float64')


,시군구,유형,지번,도로명,용도지역,건축물주용도,도로조건,전용/연면적(㎡),대지면적(㎡),거래금액(만원),층,계약년월,계약일,지분구분,건축년도,거래금액(만원)_adj,계약년월_adj,건축년도_adj
0,서울특별시 강남구 개포동,집합,6**,언주로,제3종일반주거,제2종근린생활,25m이상,58.73,,"70,000",NaN,201908,1,,1984.0,70000.0,2019-08-01,1984-01-01
1,서울특별시 강남구 개포동,일반,1***,논현로,제3종일반주거,제2종근린생활,25m이상,1119.43,347,"650,000",NaN,201908,9,,2006.0,650000.0,2019-08-09,2006-01-01
2,서울특별시 강남구 개포동,집합,1**,삼성로4길,제3종일반주거,제1종근린생활,25m이상,16.56,,"47,000",NaN,201908,14,,1983.0,47000.0,2019-08-14,1983-01-01
3,서울특별시 강남구 개포동,집합,1**,개포로,일반상업,판매,25m미만,22.82,,"13,910",NaN,201908,20,,1984.0,13910.0,2019-08-20,1984-01-01
4,서울특별시 강남구 개포동,집합,1**,개포로,일반상업,판매,25m미만,23.08,,"14,079",NaN,201908,20,,1984.0,14079.0,2019-08-20,1984-01-01


In [8]:
df.dtypes

시군구                     object
유형                      object
지번                      object
도로명                     object
용도지역                    object
건축물주용도                  object
도로조건                    object
전용/연면적(㎡)              float64
대지면적(㎡)                 object
거래금액(만원)                object
층                      float64
계약년월                     int64
계약일                      int64
지분구분                    object
건축년도                   float64
거래금액(만원)_adj           float64
계약년월_adj        datetime64[ns]
건축년도_adj        datetime64[ns]
dtype: object